# NEISS

* **Dataset:** `neiss2017.tsv` - NEISS, again! But an old one, because the newer one is too nice.
* **Source:** https://www.cpsc.gov/Research--Statistics/NEISS-Injury-Data

## 0) Import what you need

In [9]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

%matplotlib inline

# Read in `neiss2017.tsv` and inspect your data

* Check out the first few lines, the last few lines
* See how many rows and columns it is
* Check data types to be sure they are what you think they are
* If you already know some values that should be NaN, you can use `na_values` to set them up

In [10]:
df = pd.read_csv("neiss2017.tsv", sep="\t", encoding="Latin-1")
df.head()

,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Body_Part,Diagnosis,Other_Diagnosis,Disposition,Location,Fire_Involvement,Product_1,Product_2,Narrative_1,Narrative_2,Stratum,PSU,Weight
0,170100733,01/01/2017,21,1,1.0,NaN,75.0,62.0,NaN,1.0,1.0,0.0,1207.0,0.0,21YOM WAS BOXING WITH FRIEND WHILE DRINKING AN...,. BAC= 93.6 DX HEAD INJURY AND ALCOHOL INTOXIC...,S,46.0,77.6641
1,170100734,01/01/2017,71,1,1.0,NaN,31.0,53.0,NaN,1.0,5.0,0.0,1807.0,0.0,71YOM FELL ON FLOOR IN BATHROOM. DX RIB CONTUSION,NaN,S,46.0,77.6641
2,170100735,01/01/2017,16,1,1.0,NaN,33.0,51.0,NaN,1.0,1.0,0.0,676.0,342.0,16YOM TRIPPED OVER A RUG AND FELL INTO A BURNI...,DX BURNS INVOLVING LESS THAN 10% OF BODY SURFA...,S,46.0,77.6641
3,170100737,01/01/2017,55,2,1.0,NaN,75.0,62.0,NaN,1.0,1.0,0.0,620.0,0.0,55YOF WAS STANDING ON A STEP STOOL AND LOST FO...,HEAD INJURY,S,46.0,77.6641
4,170100740,01/01/2017,86,2,1.0,NaN,79.0,58.0,NaN,1.0,1.0,0.0,1646.0,4056.0,86YOF GOT TANGLED IN COAT AND FELL BACKWARD ST...,"H. DX HEMATOMA OF SOFT TISSUE, LEFT FLANK",S,46.0,77.6641


In [11]:
df.tail()

,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Body_Part,Diagnosis,Other_Diagnosis,Disposition,Location,Fire_Involvement,Product_1,Product_2,Narrative_1,Narrative_2,Stratum,PSU,Weight
386902,180309762,12/30/2017,2,2,1.0,NaN,85.0,68.0,NaN,4.0,1.0,0.0,1931.0,0.0,2YR F INGESTED 9 OF MOTHERS GLIPIZIDE PILLS;DX...,NaN,C,20.0,4.757
386903,180309763,12/31/2017,212,2,2.0,NaN,92.0,50.0,NaN,4.0,1.0,0.0,4076.0,0.0,"12MO F PT LYING IN BED, THERE WAS A DRIVE-BY S...",UGH WALL AND HIT PT IN BED; DX AMPUTATED THUMB,C,20.0,4.757
386904,180309764,12/31/2017,218,1,1.0,NaN,81.0,57.0,NaN,4.0,1.0,0.0,1395.0,0.0,18MO M SLIPPED ON TOY;DX FEMUR FX,NaN,C,20.0,4.757
386905,180309765,12/31/2017,202,1,1.0,NaN,75.0,57.0,NaN,4.0,1.0,0.0,1842.0,0.0,2MO M DAD FELL ON STAIRS WHILE HOLDING BABY;DX...,NaN,C,20.0,4.757
386906,180309766,12/31/2017,2,2,1.0,NaN,85.0,68.0,NaN,4.0,1.0,0.0,1931.0,0.0,2YR F GOT INTO DAD'S ***;DX ACCIDENTAL DRUG IN...,NaN,C,20.0,4.757


## Remove dirty data

Last time we discovered the row where Treatment Date is missing is bad, so let's remove every row where `Treatment Date` is `NaN`.

In [12]:
df = df.dropna(subset=['Treatment_Date'])

We also didn't like babies under the age of 2. Remove anyone who is 200 or older.

* **Tip:** Removing anyone 200 or older is the same as only keeping anyone under 200
* **Tip:** Because of the bad data, `df.Age` is probably a string

In [13]:
df = df[df.Age.astype(int) < 200].copy()

If you do this correctly, `df.Age.median()` should be 26.

In [14]:
df.Age.median()

26.0

# Recoding your data

Right now `Sex` is `0`, `1`, `2`, and `Race`, `Product_1`, and a lot of other fields are secret codes. Let's fix this up! In class we did a lot of `.replace(...)` by manually typing in, but there's an easier way.

## Recode with the CSV

There's a text file called `neiss_fmt.txt` in this folder that explains what all the codes are. Read it in, paying attention to what the separator is.

* **Tip:** Don't call it `df` or your other dataframe will disappear! Maybe call it `codes` or something.

### What are all the different 'Format name's?

What do they mean?

* `PROD` is product
* `DIAG` is diagnosis
* `BDYPT` is the body part injured
* `AGELTTWO` is for the of people less than 2
* `LOC` is location
* `DISP` is disposition (outcome)
* `RACE` is the person's race
* `FIRE` is whether this was fire-related
* `GENDER` matches up with the `Sex` column

Each of these is a **separate set of codes**. 

### Look at just the codes related to `GENDER`.

* **Hint:** Make sure when you filter you don't try to filter through `df`! It's really easy to forget this dataframe is named `codes` and do something like `df[codes.name == "blah blah"]`.

### Convert the codes into a dictionary

We can use this with `.replace` if we can somehow turn it into a dictionary! In class, we used this code:

```
dict(zip(code.code, code.name))
```

to create a dictionary where the keys were the `code` column and the values were the `name` column. Try to turn **just the `GENDER` results** into a dictionary. Your result should look like this:

```
{0: 'UNKNOWN', 1: 'MALE', 2: 'FEMALE'}
```

* **Tip:** `"Starting value for format"` is the code - it's always equal the `Ending value` except for the weird name stuff.
* **Tip:** It might be useful to save your filtered dataframe into something like `gender_codes` to keep your code clean.

### Use that dictionary to decode your `Sex` column

Convert your `Sex` column into being `MALE`, `FEMALE`, and `UNKNOWN`. When you're done, your `df.Sex.value_counts()` should look like this:

```
MALE      208695
FEMALE    178203
Name: Sex, dtype: int64
```

It might have some `UNKNOWN` if you didn't do `na_values=[0...` when reading in your csv, but that's okay!

* **Hint:** Remember that when you update a column you need to save the changes back into the column
* **Hint:** You should probably save your dictionary into a new variable, maybe `gender_code_dict`

## Decode the rest of your columns!

Use `RACE` and `PROD` and `BDYPRT` to un-encode in the appropriate columns.

You'll need to do all of the same stuff you did above - filter `codes`, build the dictionary, use `.replace` - but it shouldn't be so bad if you cut and paste well (3 lines per replace, but **be careful** - if you accidentally overwrite one of your columns with the wrong data, you'll need to restart your kernel and re-run your cells).

* **Tip:** Look at the `.value_counts()` we did before for `Format name`, and compare it with your column names in your `df`. Or you can be responsible and read the codebook, but you can probably guess!

## Test your decoding:

**Did you decode things correctly?** Let's find out!

`df.Body_Part.value_counts().head(2)` should give you

```
75 - HEAD    63798
76 - FACE    32879
```

`df.Race.value_counts().head(2)` should give you

```
WHITE                     160527
BLACK/AFRICAN AMERICAN     55144
```

`df.Product_1.value_counts().head(2)` should give you

```
1842 - STAIRS OR STEPS                 30647
1807 - FLOORS OR FLOORING MATERIALS    29629
```

## Save a checkpoint!

I like that we cleaned it up, save the file as `neiss_cleaned_decoded.csv`

* **Tip:** Don't you dare let the index get saved

### Analyzing the goings-on

Find every injury involving bicycles.

* **Tip:** You're searching `Product_1`, not `Narrative`
* **Tip:** What about BICYCLE vs bicycle? Do you need to care? How can you deal with it?

Graph the age distribution of injuries involving unicycles.

What was the narrative for the oldest six people who got injured on bicycles?

* **Tip:** There's some `pd.set_option` magic that will allow you to read the whole thing without a `...`

## What's the age distribution for people injured surfing?

## Golfing?

Wait, that's real weird. **Find 5 cases of people between the ages of 8 and 10 being injured golfing**.

Okay, that makes more sense.

## Plot age histograms for surfing, golfing, and bicycling

* **Tip:** Just plot them one after the other in the same cell! These days pandas put them all into the same graphic.

If you plot them in that order, import this

```
import matplotlib.pyplot as plt
```

And add a legend like this:

```
plt.legend(['Surfing', 'Cycling', 'Golfing'])
```

**WHAT'S THAT?** You want more specific bins???? When making a histogram, instead of `bins=50`, you can say "give me bins from 0 to 100 in chunks of 5" by doing this: `bins=range(0,100,5)`

In [ ]:
# TIP: If you want to see the distribution of people injured by guns, it migt look like this:
# Get every row where the product involves "gun"
# Get the ages for those rows, and drop any ages that are missing
# Do a weird 'weights' thing that you have to do in order to get a percentage in the graph
ages = df[df.Product_1.str.contains("gun", case=False)].Age.dropna()
weights = np.ones_like(ages)/float(len(ages))
ages.hist(alpha=0.5, weights=weights)

## I'm irritated by the `Narrative_1` and `Narrative_2` columns being separate

Combine them into one column called `Narrative`. The first total narrative should be

```
21YOM WAS BOXING WITH FRIEND WHILE DRINKING AND GOT PUNCHED IN THE FACE. BAC= 93.6 DX HEAD INJURY AND ALCOHOL INTOXICATION
```

I don't have anything else for us to do with that one, I just didn't like it.

## More on products

Musical instruments cause injury every now and again. What are the musical instrument categories?

* **Tip:** "MUSICAL INSTRUMENT" is too specific, make it shorter!
* **Tip:** *WE ARE NOT INTERESTED IN WIND CHIMES AT ALL!!!! THEY ARE NOT MUSICAL INSTRUMENTS!!!!*

What parts of the body do most musical instrument injuries affect?

## What's the average age of a person injured by a corkscrew vs a bottle opener?

* **Tip:** Be careful about case sensitivity!

### Why should you not write a story about that?

You might need to check some more statistics about i.

## Save the month into a separate column

Looking at `Treatment Date`, what's the month? How can you pull it out easily/lazily?

* **Tip:** You'll want to convert it to an integer
* **Tip:** You might get an error that it doesn't understand the integer `01/01` - how can you make sure it only takes the first `01` and not include the second one?
* **Tip:** Do a `.value_counts()` to be sure you're getting the month and not the day of the year!

## What month is most popular for injuries involving air conditioners?

## How about heaters?

I would have said "space heaters" but that doesn't give any results!

## Find the 5 most dangerous products

## Find the 5 most dangerous products for women vs men

* **Tip:** This is less of a tip and more of an answer, but after your `value_counts()` you should add `.groupby(level=0).nlargest(10)` and it should work

# What is the distribution of peoples' weights?

Does that seem right? Why do you think it might look like that? (It might have something to do with the corkscrew vs. bottle openers question!)